In [51]:
%%html
<!DOCTYPE html>
<html>
<head>
<style>
h1{color: #03A062;font-size:100% ;text-align: center;}
h3{color: #03A062;}
p{color: #03A062;}
</style>
</head>
<body>

<h1>Mask Detector</h1>
<h3>Detect whether a person is wearing a mask or not. It will <b>not</b> give sensible results for pictures without a human face.</h3>
<p>Upload the Photo!</p>
</body>
</html>

In [23]:
# from fastai import __version__
# print(__version__)

In [24]:
# !jupyter serverextension enable --sys-prefix voila

In [25]:
#from fastbook import *

In [26]:
from fastai.vision.all import *
from fastai.vision.widgets import *

In [32]:
# import pathlib
# temp = pathlib.PosixPath
# pathlib.PosixPath = pathlib.WindowsPath

In [30]:
# fix failure to preserve metadata in multiprocessing in DataLoader
def _rebuild_from_type(func, type, args, dict):
    ret = func(*args).as_subclass(type)
    ret.__dict__ = dict
    return ret

@patch
def __reduce_ex__(self: TensorBase, proto):
    from fastai.torch_core import _fa_rebuild_qtensor, _fa_rebuild_tensor
    torch.utils.hooks.warn_if_has_hooks(self)
    args = (type(self), self.storage(), self.storage_offset(), tuple(self.size()), self.stride())
    if self.is_quantized: args = args + (self.q_scale(), self.q_zero_point())
    args = args + (self.requires_grad, OrderedDict())
    f = _fa_rebuild_qtensor if self.is_quantized else  _fa_rebuild_tensor
    return (_rebuild_from_type, (f, type(self), args, self.__dict__))

In [33]:
learn_inf = load_learner('export.pkl', cpu=True)

In [34]:
def on_data_change2(change):
    #lbl_pred.value = ''
    img1 = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img1.to_thumb(256,256))
    pred,pred_idx,probs = learn_inf.predict(img1)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [38]:
btn_upload = widgets.FileUpload(accept='.jpg,.jpeg',multiple=False)
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_upload.observe(on_data_change2, names=['data'])
display(VBox([btn_upload,out_pl,lbl_pred ]))